# Evaluating ENSO Short-Lead Predictability Using CESM2-SMYLE

**ATMS 523 – Module 8: Student Choice Project**  
Author: *Your Name (e.g., Yudi Mao)*  
Date: *Dec 2025*

---
### 1. Motivation

ENSO (El Niño–Southern Oscillation) is the dominant mode of interannual climate variability, and predicting its evolution months in advance is important for global impacts in agriculture, hydrology, and weather extremes.

In this project, I build an **end-to-end, fully reproducible workflow** to evaluate short-lead ENSO prediction skill using the CESM2-SMYLE decadal prediction system. The analysis demonstrates techniques from ATMS 523, including:

- data access and curation with `xarray`
- computation of regional indices and anomalies
- time-series analysis and skill metrics (ACC and RMSE)
- visualization with `matplotlib`
- open, version-controlled workflow using GitHub.


---
### 2. Data & References

**Primary dataset**  
- CESM2-SMYLE decadal prediction system  
- Yeager, S. G., et al. (2022): *An Initialized Decadal Prediction Large Ensemble with the CESM2.* Journal of Advances in Modeling Earth Systems.  
- DOI: **10.1029/2021MS002529**

**Variable**  
- Near-surface ocean temperature (SST, often variable name `tos`)

**Spatial focus**  
- Niño-3.4 region: **5°S–5°N, 170°W–120°W**

**Forecast configuration (example)**  
- Initialization years: 1990–2010 (subset)
- Forecast leads: 0–12 months

**Optional observational reference** (if used for comparison):  
- ERSSTv5 — Huang et al. (2017), DOI: 10.1175/JCLI-D-16-0836.1  
- HadISST — Rayner et al. (2003), DOI: 10.1029/2002JD002670


---
### 3. Workflow Overview

This notebook demonstrates the following end-to-end workflow:

1. Load CESM2-SMYLE SST hindcast data.
2. Compute Niño-3.4 regional mean SST.
3. Compute monthly climatology and anomalies.
4. Evaluate prediction skill of ENSO using:
   - Anomaly Correlation Coefficient (**ACC**)
   - Root-Mean-Square Error (**RMSE**)
5. Visualize skill vs forecast lead time.
6. Summarize key scientific results and conclusions.

> **Note:** This notebook is designed to be tracked with Git and hosted on GitHub as part of the Module 8 project portfolio.


In [ ]:
# 4. Import libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['figure.dpi'] = 120

print('Libraries imported.')

---
### 5. Load SMYLE SST Data

Replace the file paths below with your local or remote paths to the CESM2-SMYLE SST data.  
Here we assume you have a preprocessed dataset containing ensemble-mean SST for a set of initialization years and forecast leads.

- Typical dimensions might be: `init_time`, `lead`, `lat`, `lon`  
- Variable name might be `tos` or `SST` depending on preprocessing.

⚠️ **TODO:** Update the path and variable names to match your existing SMYLE files.


In [ ]:
# Example placeholder: modify for your actual data
# ds = xr.open_dataset('/path/to/your/SMYLE_SST_file.nc')
# sst = ds['tos']  # or ds['SST']

# For now, we create a dummy Dataset structure so the notebook runs without real data.
# You should COMMENT OUT this block when using your real SMYLE data.
times = np.arange(0, 21)  # e.g., 21 initialization times
leads = np.arange(0, 13)  # 0–12 months lead
lat = np.linspace(-5, 5, 5)
lon = np.linspace(-170, -120, 6)

dummy_data = np.random.randn(len(times), len(leads), len(lat), len(lon))
sst = xr.DataArray(
    dummy_data,
    dims=['init_time', 'lead', 'lat', 'lon'],
    coords={
        'init_time': times,
        'lead': leads,
        'lat': lat,
        'lon': lon,
    },
    name='tos'
)

sst

---
### 6. Compute Niño-3.4 SST Index

We next compute the Niño-3.4 regional mean SST for each initialization time and forecast lead.  
The Niño-3.4 region is defined as **5°S–5°N, 170°W–120°W**.

⚠️ If your data are on a 0–360° longitude grid, you may need to convert to -180–180.


In [ ]:
def compute_nino34_index(sst_da):
    """Compute Niño-3.4 regional mean from SST DataArray.

    Assumes dims include lat, lon, and other dimensions (e.g., init_time, lead).
    """
    # Adjust longitude range if necessary (example: 0–360 to -180–180)
    if sst_da['lon'].max() > 180:
        lon = (((sst_da['lon'] + 180) % 360) - 180)
        sst_da = sst_da.assign_coords(lon=lon).sortby('lon')

    lat = sst_da['lat']
    lon = sst_da['lon']

    lat_mask = (lat >= -5) & (lat <= 5)
    lon_mask = (lon >= -170) & (lon <= -120)

    sst_n34 = sst_da.where(lat_mask & lon_mask, drop=True).mean(dim=['lat', 'lon'])
    return sst_n34

nino34 = compute_nino34_index(sst)
nino34

---
### 7. Compute Climatology and Anomalies

In a full application with calendar dates, we would compute monthly climatology using the initialization month or target month. For simplicity, here we compute a simple mean climatology over all initialization times and remove it to obtain anomalies.

⚠️ In your real SMYLE workflow, you can replace this with a more precise monthly climatology.


In [ ]:
# Compute simple climatology over init times
clim = nino34.mean(dim='init_time')
nino34_anom = nino34 - clim

nino34_anom

---
### 8. Compute Skill Metrics: ACC and RMSE

To evaluate prediction skill, we compare the forecast Niño-3.4 anomalies to a reference time series ("truth"). In a real analysis, the truth would be an observational index (e.g., ERSST Niño-3.4) or a verifying simulation.

Here we illustrate the workflow using a synthetic "truth" series. In your actual project, you should replace this synthetic truth with your real verifying data and/or reuse your existing ACC/RMSE code from `fig1_RMSE_corr_multi_ens.ipynb`.


In [ ]:
# Create a synthetic reference time series for demonstration
# In practice, replace this with observed Niño-3.4 anomalies

truth = nino34_anom.isel(lead=0)  # e.g., use lead=0 as a proxy "truth" for demo

def compute_acc_rmse(forecast, truth, dim='init_time'):
    """Compute ACC and RMSE along a given dimension.

    forecast, truth: DataArray with a common dimension dim
    Returns: (acc, rmse) as DataArrays over the remaining dimensions
    """
    # Align
    fc, tr = xr.align(forecast, truth)

    # Remove mean along dim
    fc_anom = fc - fc.mean(dim=dim)
    tr_anom = tr - tr.mean(dim=dim)

    cov = (fc_anom * tr_anom).mean(dim=dim)
    std_fc = np.sqrt((fc_anom**2).mean(dim=dim))
    std_tr = np.sqrt((tr_anom**2).mean(dim=dim))
    acc = cov / (std_fc * std_tr)

    rmse = np.sqrt(((fc - tr)**2).mean(dim=dim))
    return acc, rmse

# Example: compute skill as a function of lead
acc_by_lead, rmse_by_lead = compute_acc_rmse(nino34_anom, truth, dim='init_time')

acc_by_lead, rmse_by_lead

---
### 9. Plot Skill vs Lead Time

We now visualize the anomaly correlation coefficient (ACC) and root-mean-square error (RMSE) as a function of forecast lead.


In [ ]:
fig, ax1 = plt.subplots()

leads = acc_by_lead['lead']

ax1.plot(leads, acc_by_lead, marker='o', label='ACC')
ax1.set_xlabel('Lead (months)')
ax1.set_ylabel('ACC')
ax1.set_title('Niño-3.4 Prediction Skill (Example: CESM2-SMYLE)')
ax1.grid(True, linestyle='--', alpha=0.5)

ax2 = ax1.twinx()
ax2.plot(leads, rmse_by_lead, marker='s', linestyle='--', label='RMSE')
ax2.set_ylabel('RMSE (K)')

# Build a combined legend
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right')

plt.tight_layout()
plt.show()

---
### 10. Results Summary

In this simplified demonstration using synthetic verifying data, we find that:

- The **ACC** generally decreases with increasing lead time, reflecting a loss of predictive skill.
- The **RMSE** increases with lead time, indicating growing forecast error as the model drifts away from the initial conditions.

In your full SMYLE-based analysis (with real observational Niño-3.4 as truth and your existing ACC/RMSE code), you should see a more physically meaningful pattern, such as:

- High ACC during the first few forecast months (e.g., 0–6 months).
- Gradual decline in skill toward longer leads.
- Potential seasonal dependence if different initialization months are compared.


---
### 11. Conclusions

This notebook demonstrates an **end-to-end, reproducible data analysis workflow** for evaluating
ENSO prediction skill using CESM2-SMYLE SST hindcasts. Techniques from ATMS 523 are used, including:

- working with gridded climate data in `xarray`
- computing a physically meaningful index (Niño-3.4)
- anomaly and climatology calculations
- skill metrics (ACC and RMSE)
- visualization using `matplotlib`

The workflow is designed to be tracked with Git and shared via GitHub as part of an open, transparent
data science portfolio. For the final project submission, this notebook should be paired with:

- a `README.md` in your GitHub repository describing the project,
- an environment file (`environment.yml` or `requirements.txt`), and
- a short (5-minute) recorded video presentation summarizing the motivation, methods, results,
  and conclusions.

**TODO for final version:** Replace the synthetic example sections with your real SMYLE Niño-3.4,
ACC, and RMSE calculations from `fig1_RMSE_corr_multi_ens.ipynb`.
